In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,SimpleRNN,LSTM,Dropout
from keras.models import load_model

2022-09-26 13:54:13.190627: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-26 13:54:13.190656: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
raw_data=pd.read_csv('APPLE.csv')
raw_data.drop("Date",axis=1,inplace=True)
raw_data.drop("Adj Close",axis=1,inplace=True)
raw_data['Day_Avg']=(0.5*raw_data['High']+0.5*raw_data['Low'])
raw_data.head()


,High,Low,Open,Close,Volume,Day_Avg
0,40.287498,39.632500,40.215000,39.657501,114446000.0,39.959999
1,40.512501,39.972500,40.125000,40.375000,126323200.0,40.242500
2,40.990002,39.692501,40.652500,40.215000,286856000.0,40.341251
3,39.990002,39.477501,39.967499,39.912498,179629600.0,39.733751
4,39.849998,39.522499,39.747501,39.570000,95042800.0,39.686249


In [3]:
Moving_avg_7=raw_data['Close'].rolling(7).mean()
Moving_avg_50=raw_data['Close'].rolling(50).mean()
Moving_avg_100=raw_data['Close'].rolling(100).mean()

training_size=int(0.6*len(raw_data))
new_data=raw_data['Close']

x_train=raw_data.iloc[0:training_size-1]
y_train=new_data[1:training_size]
x_test=raw_data.iloc[training_size:len(raw_data)-1]
y_test=new_data[training_size+1:len(raw_data)]

#model_forest=RandomForestRegressor(n_estimators=200,criterion='mse',min_samples_split=5,max_depth=10)
model_forest=LinearRegression()
model_forest.fit(x_train,y_train)



LinearRegression()

In [4]:
def create_dataset(dataset,time_care=60):
    x_data,y_data=[],[]
    for i in range(len(dataset)-time_care-1):
        x=dataset[i:i+time_care,0]
        y=dataset[(i+time_care),0]
        x_data.append(x)
        y_data.append(y)
        
    return np.array(x_data),np.array(y_data)


In [5]:
def Deep_Predictor(col,raw_data,time_care=100,epochs=50,redefine_model=0):
    name_of_predictor=col+"_Predictor.h5"
    feature_data=raw_data[col][:len(raw_data[col])]
    Scaler=MinMaxScaler(feature_range=(0,1))
    feature_data=Scaler.fit_transform(np.array(feature_data).reshape(-1,1))
    training_size=int(len(feature_data)*0.67)
    test_size=int(len(feature_data)-training_size)
    train_data,test_data=feature_data[0:training_size:1],feature_data[training_size:len(feature_data):1]
    x_train,y_train=create_dataset(train_data,time_care)
    x_test,y_test=create_dataset(test_data,time_care)
    
    if (redefine_model==1):
        model=Sequential()
        model.add(LSTM(time_care,return_sequences=True,activation='relu',input_shape=(time_care,1)))
        model.add(LSTM(int(1.25*time_care),activation='relu',return_sequences=True))
        model.add(Dropout(0.2))

        model.add(LSTM(int(1.5*time_care),return_sequences=True,activation='relu'))
        model.add(LSTM(int(1.75*time_care),activation='relu',return_sequences=True))
        model.add(Dropout(0.3))

        model.add(LSTM(int(2*time_care),activation='relu'))
        model.add(Dense(1))
        model.compile(loss='MeanAbsolutePercentageError',optimizer='adam')    

        model.summary()
        model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=epochs,batch_size=32,verbose=1)
        model.save(name_of_predictor)
        
        train_predict=model.predict(x_train)
        train_predict=Scaler.inverse_transform(train_predict)
    
    else :
        model=load_model(name_of_predictor,compile=False)
        
    test_predict=model.predict(x_test)
    test_predict=Scaler.inverse_transform(test_predict)
     
    return Scaler.scale_

In [ ]:
redefine_model=1
scale=[]

for col in raw_data.columns:
    scale.append(Deep_Predictor(col,raw_data,time_care=240,epochs=25,redefine_model=redefine_model))

2022-09-25 14:50:22.535013: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-25 14:50:22.535045: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-25 14:50:22.535065: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (parrot): /proc/driver/nvidia/version does not exist
2022-09-25 14:50:22.535270: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 240, 240)          232320    
                                                                 
 lstm_1 (LSTM)               (None, 240, 300)          649200    
                                                                 
 dropout (Dropout)           (None, 240, 300)          0         
                                                                 
 lstm_2 (LSTM)               (None, 240, 360)          951840    
                                                                 
 lstm_3 (LSTM)               (None, 240, 420)          1312080   
                                                                 
 dropout_1 (Dropout)         (None, 240, 420)          0         
                                                                 
 lstm_4 (LSTM)               (None, 480)               1

 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 240, 240)          232320    
                                                                 
 lstm_11 (LSTM)              (None, 240, 300)          649200    
                                                                 
 dropout_4 (Dropout)         (None, 240, 300)          0         
                                                                 
 lstm_12 (LSTM)              (None, 240, 360)          951840    
                                                                 
 lstm_13 (LSTM)              (None, 240, 420)          1312080   
                                                                 
 dropout_5 (Dropout)         (None, 240, 420)          0         
                                                                 
 lstm_14 (LSTM)              (None, 480)               1729920   
                                                                 
 dense_2 (

In [ ]:
predictions_test=(model_forest.predict(x_test)).tolist()
predictions_train=(model_forest.predict(x_train)).tolist()
print(mean_squared_error(y_test,predictions_test))

In [ ]:
#plotting=predictions_train+predictions_test
plt.plot(raw_data['Close'])
plt.plot(Moving_avg_7)
plt.plot(Moving_avg_50)
plt.plot(Moving_avg_100)
plt.show()

# Future Predictor

In [ ]:
future_days=10
future_x=[]
future_y=[]
time_care=240

model_close=load_model("Close_Predictor.h5",compile=False)
model_open=load_model("Open_Predictor.h5",compile=False)
model_volume=load_model("Volume_Predictor.h5",compile=False)
model_high=load_model("High_Predictor.h5",compile=False)
model_low=load_model("Low_Predictor.h5",compile=False)
model_day_avg=load_model("Day_Avg_Predictor.h5",compile=False)

for i in range (future_days):
    x=[]
    data_considered=raw_data.iloc[len(raw_data)-time_care:len(raw_data),:]
    close=model_close.predict(np.array(scale[3]*data_considered['Close']).reshape(1,-1))
    openn=model_open.predict(np.array(scale[2]*data_considered['Open']).reshape(1,-1))
    volume=model_volume.predict(np.array(scale[4]*data_considered['Volume']).reshape(1,-1))
    high=model_high.predict(np.array(scale[0]*data_considered['High']).reshape(1,-1))
    low=model_low.predict(np.array(scale[1]*data_considered['Low']).reshape(1,-1))
    day_avg=model_day_avg.predict(np.array(scale[5]*data_considered['Day_Avg']).reshape(1,-1))
    print(close,openn,high,low,volume,day_avg)
    
    x.append(float(high/scale[0]))
    x.append(float(low/scale[1]))
    x.append(float(openn/scale[2]))
    x.append(float(close/scale[3]))
    x.append(float(volume/scale[4]))
    x.append(float(day_avg/scale[5]))
    
    series=pd.Series(x,index=raw_data.columns)
    close_linear=model_forest.predict(raw_data.iloc[len(raw_data)-1:len(raw_data),:])
    last_few_days=np.mean(data_considered.iloc[len(data_considered)-7:len(data_considered),3])
    closing_price_final=(0.4*(float(close/scale[3])) + float(0.5*close_linear) +0.1*last_few_days)
    future_y.append(closing_price_final)   
    raw_data=raw_data.append(series,ignore_index=True)
    


In [ ]:
Moving_avg_7=raw_data['Close'].rolling(7).mean()
Moving_avg_50=raw_data['Close'].rolling(50).mean()
Moving_avg_100=raw_data['Close'].rolling(100).mean()


plotting=predictions_train+predictions_test+future_y
plt.plot(Moving_avg_7)
plt.plot(Moving_avg_50)
plt.plot(Moving_avg_100)
plt.plot(plotting)
plt.show()

In [ ]:
raw_data.shape

In [ ]:
raw_data.tail()